In [ ]:
from google.colab import drive
drive.mount('/content/gdrive' , force_remount=True)
import sys

Mounted at /content/gdrive


In [ ]:
pip install pyLDAvis

In [ ]:
import pandas as pd
# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt

from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text 
%matplotlib inline

In [ ]:
tweet_filtered = pd.read_csv('/content/gdrive/My Drive/gmf/filtered_tweet_en' , engine = 'python')


In [ ]:
tweet_filtered.tweet = tweet_filtered.tweet.str.lower()

In [ ]:
topic_keywords_filtered = tweet_filtered[((tweet_filtered['tweet'].str.contains('turkey')) | (tweet_filtered['tweet'].str.contains('turkish')))
                                         
                                         & ((tweet_filtered['tweet'].str.contains('american')) | (tweet_filtered['tweet'].str.contains(r'(?:\s|^)us(?:\s|$)')))
                                         ]
topic_keywords_filtered = topic_keywords_filtered.reset_index(drop=True)
len(topic_keywords_filtered)

2412

In [ ]:
topic_keywords_filtered.head()

,id,date,time,user_id,username,tweet,hashtags,retweet,user_rt,mentions
0,1236382327154098176,2020-03-07,21:04:52,3.031094e+09,m_suchkov,"said halim pasha (r),grand vizier of ottoman e...","['#turkey', '#erdogan', '#russia']",False,NaN,[]
1,1205197666231169027,2019-12-12,19:48:09,3.031094e+09,m_suchkov,breaking: #us senate unanimously recognizes #a...,"['#us', '#armeniangenocide', '#turkey']",False,NaN,[]
2,1183355378723164167,2019-10-13,14:14:42,3.031094e+09,m_suchkov,"just in: as #turkish-led forces advance, #us t...","['#turkish', '#us']",False,NaN,[]
3,1183164485277835267,2019-10-13,01:36:10,3.031094e+09,m_suchkov,just in: #us forces say #turkey was deliberate...,"['#us', '#turkey', '#syria']",False,NaN,[]
4,1184522454556909568,2019-10-16,19:32:15,4.540999e+08,mcfaul,mr. @realdonaldtrump when say you didn't give ...,[],False,NaN,['realdonaldtrump']


In [ ]:
topic_keywords_filtered['tweet'][0]

'said halim pasha (r),grand vizier of ottoman empire\xa0(1913/17) \ntravels in time to take power in modern day #turkey as prez #erdogan (l) to restore tur might\n\nmeanwhile native american chief from kaw/kansa tribe (r) travels in time as prez of #russia to continue fighting americans pic.twitter.com/vcqkefsrti'

In [ ]:
import nltk
import string
nltk.download('punkt')
nltk.download('stopwords')


user_defined_stop_words = ['especially' , 'getting' , 'according', 'without','years', 'take', 'use', 'point', 'since', 
                           'years', 'year', 'the' ,'https' , 'http' ,"''", "``" , "’" , "“" , "n't" , "'d", 'com',
                           'status', 'pic', 'twitter', 'www', 'news' , 'org', 'ly' , 'bit' , 'bitly', 'html' , 'org',
                           'co' , 'know' , 'yes' , 'even', 'much' , 'really' , 'know' , 'well', 'much' , 'going', "really",
                           "always" , "sure", "read" , "things", "today" , "last", "new" , "read" , 'right', 'great' , 
                           'point',"one","would" ,"via" , "…" ,"”" ,  "..." ,"'s" , "time", "says" , "like", 
                           "good", "also", "get" , "said", "could", "say", "much","even", "many", "may" , "still" , 
                           "--" , "'m", "'re", '📸', "want" , "another" , "piece", "let" , "made", "got","'ve" , 
                           'seems' , 'look' ,  "'ll" ,"rt" , "'‘'", "‘", "ca", "actually" , "'—'" , "—", 
                           'ever' , 'lot' , 'already' , "maybe", 'used', 'though' , '..', 'likely' , 'nothing' ,
                           "people" , "know" "right", "see" , "really" , "day" , "well", "much", "going" , "even" , 
                           "yes" , "way" , "never" , "go" , "something" , "thing" , "make" , "back"  , 'watch' , 
                           'youtube', 'one' , "the" , "to",  "da" ,"is" ,"for" ] 
i = nltk.corpus.stopwords.words('english')
j = list(string.punctuation) + user_defined_stop_words
stopwords = set(i).union(j) 
stopwords_en = list(stopwords)   

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def print_topics(model, count_vectorizer, n_top_words):
    # get the word names from the BOW count vector
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        # topic is a vector of probabilities for each word from the corpus
        # .argsort() returns a list of indezes to access in that order to obtain a sorted (ascending) version of the topic array
        idxs = topic.argsort()
        # reverse the list to obtain descending sort order -- the most important keywords (their indizes) are now in front
        idxs = idxs[::-1]
        # get the top words for each topic from the BOW vector
        print(" ".join([words[i] for i in idxs[:n_top_words]]))

In [ ]:
# Initialise the count vectorizer with the English stop words
count_vectorizer = CountVectorizer(stop_words = stopwords_en)

# Fit and transform the processed titles
count_data = count_vectorizer.fit_transform(topic_keywords_filtered.tweet)

number_topics = 4
number_words = 15

lda = LDA(n_components=number_topics)
lda.fit(count_data)

# Print the topics found by the LDA model
print(" 4 Topics found via LDA:")
print_topics(lda, count_vectorizer, number_words)

 4 Topics found via LDA:

Topic #0:
us turkey turkish american world syria state security erdogan iraq coup tr relations gulen military

Topic #1:
us turkey turkish syria american isis russia sanctions trump syrian policy iran border foreign officials

Topic #2:
us turkey syria turkish russia erdogan ypg trump american syrian kurds pkk forces zone president

Topic #3:
turkey us turkish american syria deal sanctions nato defense russian ypg iran minister trump 400


In [ ]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda, count_data , count_vectorizer, mds='tsne', sort_topics = True)
panel


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2      269.562408 -124.803398       1        1  46.186112
1        1.782994 -295.079681       2        1  22.755475
0      220.811432 -343.832123       3        1  16.968095
3       50.533993  -76.050934       4        1  14.090318, topic_info=            Term         Freq        Total Category  logprob  loglift
9941      turkey  1969.000000  1969.000000  Default  30.0000  30.0000
10171         us  2284.000000  2284.000000  Default  29.0000  29.0000
3401        deal    88.000000    88.000000  Default  28.0000  28.0000
1701    american   273.000000   273.000000  Default  27.0000  27.0000
10704       zone    87.000000    87.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
1708   americans    10.245780    45.927724   Topic4  -6.5040   0.4595
6701    military    11.695554   121.711085   Topic4  -6.3717  -0.3828
8772    security    10.499733    86.518750   Topic4  -6.4795  -0.1493
7479      pastor     9.552424    44.616734   Topic4  -6.5741   0.4184
8736         sdf     9.635242    80.130993   Topic4  -6.5655  -0.1586

[332 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
1          1  0.223392   000
1          2  0.726023   000
1          3  0.055848   000
22         1  0.495836    07
22         2  0.148751    07
...      ...       ...   ...
10660      1  0.798821   ypg
10660      2  0.100514   ypg
10660      4  0.100514   ypg
10704      1  0.900468  zone
10704      4  0.102585  zone

[582 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1, 4])

In [ ]:
pyLDAvis.save_html(panel, '/content/gdrive/My Drive/gmf/solutions/second_filtered_keyword_en_4k'+'.html')

In [ ]:
import pandas as pd
tweet_filtered_tr = pd.read_csv('/content/gdrive/My Drive/gmf/filtered_tweet_tr' , engine='python')

In [ ]:
tweet_filtered_tr.tweet = tweet_filtered_tr.tweet.str.lower()

In [ ]:
topic_keywords_filtered_tr = tweet_filtered_tr[((tweet_filtered_tr['tweet'].str.contains('türkiye')) | (tweet_filtered_tr['tweet'].str.contains('türk')))
                                         
                                         & ((tweet_filtered_tr['tweet'].str.contains('abd')) | (tweet_filtered_tr['tweet'].str.contains('amerika')))
                                         ]
topic_keywords_filtered_tr = topic_keywords_filtered_tr.reset_index(drop=True)
len(topic_keywords_filtered_tr)


9865

In [ ]:
topic_keywords_filtered_tr['tweet'][0]

'• dışarı çıkarken hep maske taktığını söyleyenlerin oranı türkiye’de %70-80 civarında, dünya için makul bir ortalama\n• salgından en çok etkilenen abd ve i̇ngiltere gibi ülkelerde oranın düşük olmasıysa akıl kârı değil pic.twitter.com/rrbskdiiee'

In [ ]:
import nltk
import string
nltk.download('punkt')
nltk.download('stopwords')
from nltk import sent_tokenize, word_tokenize
user_defined_stop_words_tr = [ '”','“','son', 'ilk', '1' , '2' ,'3', 'arasında' , 'fazla' , 'yeniden' , 'bunun', 'etmek', 'dedi' , 'buna' , 'bugünkü',
 'bugün', '…', '’', '...', "''", '``', "ne" , "kadar" , "ve" , "iyi" , "ki" , "çok" , "ya" , "diye" , "gibi" , "com", 'öyle' , "önce" , 
                              "twitter" , "pic" , "ile" , "en" ,"http" , "https" , "html" , "aracılığıyla" , "için",
                              "değil", "çok" , "daha"  , "ile" , "ama" , "ne"  , "gibi" ,  "her" , 'status' , "pic", "org", "id",
                              "aspx" , "ile" , "teşekkürler" , "bu" , "youtube" , "teşekkür" , "the" , "to",  "da" ,"is" ,"for",  "eyvallah",                            
                              'http' , 'bir' , 'var' , 'yok', 'olarak' , 'mi', 'ben', 'olan', 'olsun', 'bu' , 'de' , 'da' , 'ama', 
    'nin' , 'in' , 'bile' , 'böyle' , 'olur' , 'göre' , 'cok' , 'tek' , 'iki' , 'oldu' ,  'kendi', "şimdi", 'yeni', 'bugün' , 'yazar', 'olun',
     'hocam' , 'be' , 'evet' ,  'sonra' , 'youtu',  'bence' ,'size' , 'sana' , 'zaman', "www", 'yazarlar' , 'haber' , 'via',
 'olduğu','icin', 'bunu','ilgili','artık', 'aynı' , 'a' , 'ancak' , 'ederim' , 'zaten' , 'olacak' , 'ediyor', 'gerek', 'bize',
 'yine' , 'tam' , 'başka' , 'benim' ,  'ın' , 'olduğunu' , '‘' ,  'içinde' , 'sen' , 'olmak' , 'hala', 'demek' , 'bizim',
 'eden' , 'bana' , 'bi' ,  'üzerine' , 'tarafından', 'bi', 'olsa' , 'diyor' ,  'nın' , 'ye' , 'etti', 'geldi' , 'degil',
 'uzun' , 'bin' ,'oluyor' , 'şekilde' , 'e' , 'of' , 'el' , 'i' , 'diğer',  'dair' , 'yerine' , 'geliyor' , 'olması' , 'an',
 'pek' ,  'den' , 'yoksa', 'olmaz' , 'üzerinden' , 'onu' , 'beni' , 'seni' , 'te' , 'yi' ,'karsi' , 'karşı' , "'i" , "'ı" , "yı" ,'e' , 'den' ,'dan' ,'tan',
 'ten' , 'watch' , 'tebrikler' , 'sadece' , 'and' ,  'biraz' , 'on', 'lazım' , 'tek' , 'sizin' , 'olabilir' , 'olmuş']
i = nltk.corpus.stopwords.words('turkish')
j = list(string.punctuation) + user_defined_stop_words_tr
stopwords_tr = set(i).union(j)
stopwords_tr = list(stopwords_tr)
type(stopwords_tr)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


list

In [ ]:
# Initialise the count vectorizer with the English stop words
count_vectorizer_tr = CountVectorizer(stop_words = stopwords_tr)

# Fit and transform the processed titles
count_data_tr = count_vectorizer_tr.fit_transform(topic_keywords_filtered_tr.tweet)

number_topics = 4
number_words = 15

lda_tr = LDA(n_components=number_topics)
lda_tr.fit(count_data_tr)

# Print the topics found by the LDA model
print(" 4 Topics found via LDA:")
print_topics(lda_tr, count_vectorizer_tr, number_words)

 4 Topics found via LDA:

Topic #0:
abd türkiye tr suriye türk rusya amerikan trump turkiye ran 400 ypg s400 devam askeri

Topic #1:
abd türkiye suriye c4 tr turkiye çin trump türk rusya deki ran irak abdnin hedef

Topic #2:
abd türkiye türk suriye trump amerikan nato rusya 35 400 yaptırım ran pkk turk dışişleri

Topic #3:
abd türkiye tr fikir rusya suriye pkk turkiye 21yyte hurriyet pyd tanki 21 yüzyıl enstitüsü


In [ ]:
pyLDAvis.enable_notebook()
panel_tr = pyLDAvis.sklearn.prepare(lda_tr, count_data_tr , count_vectorizer_tr, mds='tsne', sort_topics = True)
panel_tr


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2      269.562408 -124.803398       1        1  33.888536
0        1.782994 -295.079681       2        1  28.852384
3      220.811432 -343.832123       3        1  27.876748
1       50.533993  -76.050934       4        1   9.382331, topic_info=            Term         Freq        Total Category  logprob  loglift
16202      fikir   841.000000   841.000000  Default  30.0000  30.0000
2199      21yyte   557.000000   557.000000  Default  29.0000  29.0000
37339         tr  2336.000000  2336.000000  Default  28.0000  28.0000
35882      tanki   457.000000   457.000000  Default  27.0000  27.0000
20350   hurriyet   552.000000   552.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
16167       fetö    22.328449   132.550517   Topic4  -6.8891   0.5852
28511       nato    31.996956   741.809624   Topic4  -6.5294  -0.7771
14376  dışişleri    26.365300   337.589943   Topic4  -6.7230  -0.1834
5923      ankara    24.751568   248.167175   Topic4  -6.7861   0.0611
12418     destek    23.901218   295.395629   Topic4  -6.8211  -0.1480

[326 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
1          1  0.121683       000
1          2  0.243366       000
1          4  0.608414       000
32         1  0.124310        06
32         2  0.056504        06
...      ...       ...       ...
44108      2  0.265684    önemli
44108      3  0.103322    önemli
44108      4  0.062731    önemli
44911      4  0.861977  şaşırttı
45002      4  0.945695    şeydir

[558 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 2])

In [ ]:
pyLDAvis.save_html(panel_tr, '/content/gdrive/My Drive/gmf/solutions/second_filtered_keyword_TR_4k'+'.html')